In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
class RealisticBabySimulator:
    def __init__(self, baby_ids, start_date, total_weeks=8):
        self.baby_ids = baby_ids
        self.start_date = start_date
        self.total_weeks = total_weeks
        self.log_interval_seconds = 10
        self.logs_per_minute = 60 // self.log_interval_seconds
        
        # 주차별 발달 단계: (기본수면확률, 평균 낮잠 시간(분), 최대 깨어있는 시간(분), 야간수면강화, 밤중깸확률)
        # 밤중깸확률: 밤 시간 동안, 잠든 상태에서 짧게 깰(브레이크) 확률 (기본값)
        self.developmental_stages = {
            0:  {'base_sleep_prob': 0.60, 'avg_nap_duration_min': 45, 'max_awake_min': 60,  'night_sleep_factor': 1.3, 'night_break_prob': 0.15}, # 신생아
            1:  {'base_sleep_prob': 0.58, 'avg_nap_duration_min': 50, 'max_awake_min': 75,  'night_sleep_factor': 1.3, 'night_break_prob': 0.12},
            4:  {'base_sleep_prob': 0.55, 'avg_nap_duration_min': 60, 'max_awake_min': 90,  'night_sleep_factor': 1.25,'night_break_prob': 0.10}, # 1개월
            8:  {'base_sleep_prob': 0.50, 'avg_nap_duration_min': 75, 'max_awake_min': 100, 'night_sleep_factor': 1.2, 'night_break_prob': 0.08}, # 2개월
            16: {'base_sleep_prob': 0.45, 'avg_nap_duration_min': 90, 'max_awake_min': 120, 'night_sleep_factor': 1.15,'night_break_prob': 0.06}, # 4개월 (수면퇴행기 고려 필요시 별도 로직)
            24: {'base_sleep_prob': 0.40, 'avg_nap_duration_min': 120,'max_awake_min': 150, 'night_sleep_factor': 1.1, 'night_break_prob': 0.05}, # 6개월
            48: {'base_sleep_prob': 0.35, 'avg_nap_duration_min': 120,'max_awake_min': 180, 'night_sleep_factor': 1.05,'night_break_prob': 0.04}  # 12개월
        }
        self.env_profiles = self._initialize_environment_profiles()

    def _get_stage_params(self, week):
        stages = sorted(self.developmental_stages.keys(), reverse=True)
        for s_week_start in stages:
            if week >= s_week_start:
                return self.developmental_stages[s_week_start]
        return self.developmental_stages[0] # 기본값 (신생아)

    def _initialize_environment_profiles(self):
        profiles = {}
        for bid in self.baby_ids:
            profiles[bid] = {
                'temp': np.random.uniform(23.0, 27.0), 'humidity': np.random.uniform(50.0, 70.0),
                'brightness': np.random.uniform(70.0, 100.0), 'noise': np.random.randint(40, 70),
                'optimal_temp_day': np.random.normal(23.0, 1.0), 
                'optimal_temp_night': np.random.normal(21.0, 1.0),
                'adapt_speed': np.random.uniform(0.05, 0.2)
            }
        return profiles
    
    def _adjust_environment(self, baby_id, is_sleeping_now, current_time):
        profile = self.env_profiles[baby_id]
        hour = current_time.hour
        
        is_night = 21 <= hour or hour < 6 # 밤 시간 정의 (예: 21시 ~ 06시)

        if is_sleeping_now:
            temp_target = profile['optimal_temp_night'] if is_night else profile['optimal_temp_day']
            noise_target = np.random.randint(35, 45) # 수면 중엔 조용하게
            brightness_target = np.random.uniform(0, 20) # 수면 중엔 어둡게
        else: # 깨어있을 때
            temp_target = profile['optimal_temp_day'] + np.random.uniform(-1,1) # 약간의 변동성
            noise_target = np.random.randint(45, 60)
            brightness_target = np.random.uniform(60, 100) # 활동 시간에는 밝게

        profile['temp'] += (temp_target - profile['temp']) * profile['adapt_speed'] * np.random.uniform(0.8, 1.2)
        profile['noise'] = int(profile['noise'] + (noise_target - profile['noise']) * 0.2 * np.random.uniform(0.8, 1.2))
        profile['brightness'] += (brightness_target - profile['brightness']) * profile['adapt_speed'] * np.random.uniform(0.8,1.2)

        profile['temp'] = np.clip(profile['temp'], 18.0, 30.0)
        profile['humidity'] = np.clip(profile['humidity'] + np.random.normal(0, 0.5), 40.0, 75.0) # 습도는 천천히 변동
        profile['noise'] = np.clip(profile['noise'], 30, 80)
        profile['brightness'] = np.clip(profile['brightness'], 0, 100)

    def generate_data_for_baby(self, baby_id):
        """한 아기의 전체 기간 데이터를 생성"""
        baby_data = []
        log_id_start_value = baby_id * 10000000 # baby_id별 log_id 시작 값 다르게 (선택적)

        current_time = self.start_date
        end_simulation_time = self.start_date + timedelta(weeks=self.total_weeks)
        
        week = 0
        last_week_update_time = self.start_date
        
        # 아기 상태 변수
        current_is_sleeping = np.random.choice([True, False]) # 초기 상태 랜덤
        consecutive_sleep_logs = 0
        consecutive_awake_logs = 0
        
        # 낮잠 관련 변수
        naps_today_count = 0
        last_nap_end_time = current_time - timedelta(hours=4) # 초기값 (충분히 과거)

        # 메인 루프 (10초 간격)
        log_counter_for_baby = 0
        while current_time < end_simulation_time:
            if (current_time - last_week_update_time).days >= 7:
                week += 1
                last_week_update_time = current_time
            
            stage_params = self._get_stage_params(week)
            self._adjust_environment(baby_id, current_is_sleeping, current_time)
            profile = self.env_profiles[baby_id]
            
            current_hour = current_time.hour
            is_night_period = 21 <= current_hour or current_hour < 7 # 밤잠 시간대 (예: 21시~07시)

            # --- is_sleeping 상태 결정 로직 ---
            prob_to_sleep = stage_params['base_sleep_prob']
            prob_to_wake = 1 - prob_to_sleep

            # 1. 시간대별 조정
            if is_night_period:
                prob_to_sleep *= stage_params['night_sleep_factor']
            else: # 낮 시간
                prob_to_sleep *= 0.7 # 밤보다는 덜 잠

            # 2. 연속 각성/수면 시간 기반 조정
            if consecutive_awake_logs > stage_params['max_awake_min'] * self.logs_per_minute:
                prob_to_sleep = min(1.0, prob_to_sleep * 1.5) # 오래 깨있으면 잠들 확률 증가
            
            max_continuous_sleep_logs_day = stage_params['avg_nap_duration_min'] * 1.5 * self.logs_per_minute # 최대 낮잠 시간
            max_continuous_sleep_logs_night = 8 * 60 * self.logs_per_minute # 최대 밤잠 (예: 8시간)
            
            if current_is_sleeping:
                if is_night_period and consecutive_sleep_logs > max_continuous_sleep_logs_night:
                    prob_to_wake = min(1.0, prob_to_wake * 2.0) # 너무 오래 자면 깰 확률 증가 (밤)
                elif not is_night_period and consecutive_sleep_logs > max_continuous_sleep_logs_day:
                     prob_to_wake = min(1.0, prob_to_wake * 2.0) # 너무 오래 자면 깰 확률 증가 (낮)


            # 3. 상태 전환 결정 (매 분 또는 특정 조건마다)
            # 여기서는 더 자주 상태를 평가하도록 매 10초마다 확률 기반으로 결정
            rand_val = np.random.rand()

            if current_is_sleeping: # 현재 자고 있다면
                # 밤이고, 아주 낮은 확률로 "브레이크" 발생 (1~4분 False)
                if is_night_period and rand_val < stage_params['night_break_prob'] / (60 * self.logs_per_minute / 10) : # 10분당 1회 정도의 확률로 조정
                    break_duration_logs = np.random.randint(1 * self.logs_per_minute, 4 * self.logs_per_minute + 1)
                    for _ in range(break_duration_logs):
                        if current_time >= end_simulation_time: break
                        baby_data.append(self._create_log_entry(log_id_start_value + log_counter_for_baby, baby_id, False, current_time, profile, week))
                        log_counter_for_baby +=1
                        current_time += timedelta(seconds=self.log_interval_seconds)
                        if current_time >= end_simulation_time: break
                    # 브레이크 후에는 다시 잠든다고 가정 (또는 확률적으로 결정)
                    current_is_sleeping = True 
                    consecutive_sleep_logs = 0 # 브레이크 후 수면 시간 리셋
                    consecutive_awake_logs = 0
                    if current_time >= end_simulation_time: break
                    continue # 다음 10초 루프로

                # 일반적인 깸 확률
                if rand_val < prob_to_wake / (10 * self.logs_per_minute) : # 10분당 한번꼴 확률로 조정
                    current_is_sleeping = False
            
            else: # 현재 깨어 있다면
                if rand_val < prob_to_sleep / (10 * self.logs_per_minute) :
                    # "완전 깸" (5분 이상 False) 시나리오를 만들기 위해,
                    # 깨어있는 상태가 일정 시간 지속되도록 함.
                    # 여기서는 일단 확률로 잠들게 하지만,
                    # 실제로는 깨어난 후 바로 잠들지 않는 패턴이 더 현실적.
                    # 이 부분은 개선 여지 있음 (예: 최소 각성 시간 보장)
                    current_is_sleeping = True
            
            # 상태 지속 시간 업데이트
            if current_is_sleeping:
                consecutive_sleep_logs += 1
                consecutive_awake_logs = 0
            else:
                consecutive_awake_logs += 1
                consecutive_sleep_logs = 0

            # --- "완전 깸" (5분 이상 False) 시나리오 만들기 ---
            # 만약 깨어난 상태(current_is_sleeping=False)이고,
            # consecutive_awake_logs 가 (5분 * logs_per_minute) 보다 작다면,
            # 강제로 계속 깨어있도록 할 수도 있음. (하지만 이건 너무 인위적일 수 있음)
            # 여기서는 확률 기반으로 하되, "연속 각성 시간 기반 조정"에서 잠들 확률을 낮추는 식으로 간접 유도.

            # 로그 생성
            baby_data.append(self._create_log_entry(log_id_start_value + log_counter_for_baby, baby_id, current_is_sleeping, current_time, profile, week))
            log_counter_for_baby +=1
            current_time += timedelta(seconds=self.log_interval_seconds)

        return baby_data

    def _create_log_entry(self, log_id, baby_id, is_sleeping, timestamp, profile, week):
        return {
            "log_id": log_id, "baby_id": baby_id, "is_sleeping": is_sleeping,
            "created_at": timestamp,
            "temperature": round(profile['temp'], 1), "humidity": round(profile['humidity'], 1),
            "brightness": round(profile['brightness'], 1), "white_noise_level": profile['noise'],
            "week": week
        }

    def generate_all_data(self):
        all_data = []
        for baby_id_val in self.baby_ids:
            print(f"Simulating data for baby ID: {baby_id_val}...")
            all_data.extend(self.generate_data_for_baby(baby_id_val))
        return pd.DataFrame(all_data)

In [3]:
simulator = RealisticBabySimulator(baby_ids=[1, 2, 3, 4, 5], 
                                   start_date=datetime(2024, 5, 27), 
                                   total_weeks=48) # 또는 테스트를 위해 total_weeks=1 등으로 짧게
df_realistic_dummy = simulator.generate_all_data()

Simulating data for baby ID: 1...
Simulating data for baby ID: 2...
Simulating data for baby ID: 3...
Simulating data for baby ID: 4...
Simulating data for baby ID: 5...


In [4]:
df_realistic_dummy.to_csv('2nd_dummy_baby_sleep_data.csv', index=False, encoding='utf-8-sig')

In [5]:
print(df_realistic_dummy.head())
print(f"\nTotal logs generated: {len(df_realistic_dummy)}")

     log_id  baby_id  is_sleeping          created_at  temperature  humidity  \
0  10000000        1        False 2024-05-27 00:00:00         25.0      66.3   
1  10000001        1        False 2024-05-27 00:00:10         24.8      66.3   
2  10000002        1         True 2024-05-27 00:00:20         24.6      65.2   
3  10000003        1         True 2024-05-27 00:00:30         24.0      65.4   
4  10000004        1         True 2024-05-27 00:00:40         23.4      65.5   

   brightness  white_noise_level  week  
0        78.6                 49     0  
1        80.5                 48     0  
2        78.1                 48     0  
3        68.1                 45     0  
4        60.0                 44     0  

Total logs generated: 14515200


In [6]:
# 생성된 데이터에서 is_sleeping 분포 확인
print("\nIs Sleeping Distribution:")
print(df_realistic_dummy['is_sleeping'].value_counts(normalize=True))


Is Sleeping Distribution:
is_sleeping
False    0.560444
True     0.439556
Name: proportion, dtype: float64


**주요 수정 방향:**

1.  **`is_sleeping` 상태의 동적 결정:** 이것이 가장 핵심입니다. 아기의 연령(주차), 시간대(밤/낮), 연속 수면/각성 시간 등을 고려하여 `is_sleeping` 상태가 변하도록 합니다.
2.  **"완전 깸" 및 "브레이크" 시나리오 명시적 생성:** 분석 함수가 감지해야 할 주요 이벤트를 더미 데이터에 포함시킵니다.
3.  **낮잠 로직 구체화:** `nap_count`를 활용하여 현실적인 낮잠 패턴을 시도합니다.
4.  **Wake Window (깨어있는 시간) 고려:** 아기가 특정 시간 이상 깨어있으면 잠들 확률을 높이는 로직을 추가합니다.
5.  **수면 주기(Sleep Cycle) 내의 짧은 각성(브레이크) 시뮬레이션:** 깊은 잠과 얕은 잠 사이의 전환점에서 발생할 수 있는 짧은 각성을 표현합니다.

**주요 변경 및 추가 사항:**

1.  **클래스명 변경:** `RealisticBabySimulator`
2.  **`developmental_stages` 상세화:**
    *   `avg_nap_duration_min` (평균 낮잠 시간)
    *   `max_awake_min` (최대 깨어있는 시간 - Wake Window 개념)
    *   `night_sleep_factor` (야간 수면 강화 계수)
    *   `night_break_prob` (밤중 짧은 깸 발생 확률)
3.  **`generate_data_for_baby` 메서드 분리:** 각 아기별 데이터 생성을 별도 메서드로 만들어 관리 용이.
4.  **`is_sleeping` 상태 결정 로직:**
    *   **상태 변수:** `current_is_sleeping`, `consecutive_sleep_logs`, `consecutive_awake_logs`를 사용하여 현재 상태와 지속 시간을 추적.
    *   **확률 기반 전환:**
        *   기본 수면 확률(`base_sleep_prob`)을 설정하고, 시간대(밤/낮), 연속 각성/수면 시간에 따라 이 확률을 조정.
        *   매 10초마다 또는 특정 주기마다 랜덤 값과 조정된 확률을 비교하여 상태 전환 여부 결정. (현재는 매 10초마다 확률적으로 상태를 평가하고, 실제 전환은 더 낮은 빈도로 발생하도록 조정)
    *   **밤중 "브레이크" 시뮬레이션:**
        *   밤 시간 동안 `current_is_sleeping == True`일 때, `night_break_prob`에 따라 1~4분 동안 `is_sleeping = False`로 잠시 전환되는 "브레이크"를 명시적으로 생성. 브레이크 후에는 다시 잠든다고 가정 (또는 확률적으로 결정).
    *   **"완전 깸" 유도:**
        *   `current_is_sleeping == False`일 때, 바로 다시 잠들지 않도록 `prob_to_sleep`을 낮추거나, 최소 각성 시간을 보장하는 로직을 추가할 수 있습니다 (현재 코드는 확률 기반이며, 이 부분은 더 개선할 수 있습니다).
        *   "너무 오래 자면 깰 확률 증가" 로직을 통해 자연스러운 깸을 유도.
5.  **`_create_log_entry` 메서드:** 로그 생성 부분을 함수로 분리.
6.  **`generate_all_data` 메서드:** 모든 아기의 데이터를 생성하고 하나의 DataFrame으로 합치는 최상위 메서드.

**추가 고려 및 개선 사항:**

*   **"완전 깸" (5분 이상 False) 패턴 강화:** 현재 로직은 확률 기반이라 5분 이상 연속 False가 충분히 자주 발생하지 않을 수 있습니다. 특정 조건(예: 밤잠 종료 시, 긴 낮잠 후)에서 의도적으로 5분 이상 깨어있는 기간을 만드는 것을 고려할 수 있습니다.
*   **낮잠 스케줄링:** `nap_count`를 직접적으로 사용하여 하루에 특정 횟수의 낮잠을 자도록 유도하는 로직을 추가하면 더 현실적일 것입니다. (예: 아침에 깨어난 후 일정 시간 뒤 첫 낮잠, 그 후 Wake Window를 거쳐 다음 낮잠 등)
*   **수면 압력(Sleep Pressure) 모델링:** 깨어있는 시간이 길어질수록 잠들 확률이 높아지고, 자는 시간이 길어질수록 깰 확률이 높아지는 것을 더 정교하게 모델링할 수 있습니다.
*   **4개월 수면 퇴행 등 특정 이벤트:** 특정 주차에 수면 패턴이 더 불규칙해지는 현상을 시뮬레이션에 추가할 수 있습니다.

이 수정된 시뮬레이터로 생성된 데이터는 `is_sleeping` 상태가 동적으로 변하고, "브레이크"와 (이론적으로) "완전 깸" 패턴을 포함할 가능성이 훨씬 높습니다. 이 데이터를 사용하여 `analyze_sleep_cycle_details_v2` 함수를 다시 테스트해보시면 이전과는 다른, 더 의미 있는 결과를 얻으실 수 있을 것입니다.

생성된 데이터의 `is_sleeping` 분포를 확인하고, 특정 날짜의 데이터를 시각화해보는 것도 데이터가 잘 생성되었는지 판단하는 데 도움이 됩니다.

In [7]:
from datetime import timedelta, datetime
import logging # 로깅 라이브러리 사용

In [8]:
# 로거 설정 (필요한 경우에만 상세 로깅)
# logging.basicConfig(level=logging.INFO) # INFO 레벨 이상 모두 출력
# logger = logging.getLogger(__name__)

def analyze_sleep_cycle_details_v2(daily_logs_df, # 함수 이름 변경 (버전 관리)
                                   log_interval_seconds=10,
                                   min_sleep_duration_for_state_minutes=5,
                                   sleep_state_true_threshold_ratio=0.8,
                                   break_min_duration_minutes=1,
                                   awakening_min_duration_minutes=5,
                                   min_valid_chunk_duration_minutes=10,
                                   debug_date=None): # 특정 날짜 디버깅용 파라미터
    """
    하루 동안의 로그 데이터를 분석하여 주요 수면 사이클(청크)을 식별하고,
    각 청크 내의 상세 정보(실제 잠든 시간, 깬 시간, 브레이크 횟수, 수면 효율 등)를 추출합니다.
    (이전 버전에서 Warning 처리 및 디버깅 옵션 추가)
    """
    if daily_logs_df.empty:
        return []

    # 현재 날짜 (디버깅용)
    current_processing_date = None
    if not daily_logs_df.empty:
        current_processing_date = daily_logs_df['created_at'].iloc[0].date()


    if not pd.api.types.is_datetime64_any_dtype(daily_logs_df['created_at']):
        daily_logs_df['created_at'] = pd.to_datetime(daily_logs_df['created_at'])
    daily_logs_df = daily_logs_df.sort_values(by='created_at').reset_index(drop=True)
    daily_logs_df['is_sleeping_int'] = daily_logs_df['is_sleeping'].astype(int)

    logs_per_minute = 60 // log_interval_seconds
    min_logs_for_state = min_sleep_duration_for_state_minutes * logs_per_minute
    break_min_logs = break_min_duration_minutes * logs_per_minute
    awakening_min_logs = awakening_min_duration_minutes * logs_per_minute

    sleep_cycles = []
    current_log_index = 0
    num_logs = len(daily_logs_df)

    while current_log_index < num_logs:
        potential_sleep_start_index = -1
        actual_fell_asleep_time = None

        for i in range(current_log_index, num_logs - min_logs_for_state + 1):
            window = daily_logs_df.iloc[i : i + min_logs_for_state]
            if window['is_sleeping_int'].mean() >= sleep_state_true_threshold_ratio:
                potential_sleep_start_index = i
                actual_fell_asleep_time = window.iloc[0]['created_at']
                break
        
        if potential_sleep_start_index == -1:
            break

        chunk_processing_start_index = potential_sleep_start_index # '잠듦' 판단 윈도우 시작
        chunk_actual_start_time = actual_fell_asleep_time # 실제 '잠듦' 시간 (윈도우 첫 로그)

        breaks_in_chunk = 0
        is_false_streak_count = 0 # 각 청크 시작 시 리셋
        
        # `actual_fell_asleep_time` 이후부터 완전 깸 탐색 시작
        # `potential_sleep_start_index`는 윈도우의 시작이므로, 실제 탐색은 윈도우 끝 이후부터.
        # 또는, '잠듦'으로 판단된 첫 로그부터 is_sleeping 상태를 추적해도 됨.
        # 여기서는 '잠듦'으로 판단된 윈도우의 첫 로그부터 상태를 추적.
        # 즉, k는 potential_sleep_start_index 부터 시작.
        
        # 이 루프는 '잠듦'이 확인된 시점부터 데이터의 끝까지 또는 '완전 깸'이 확인될 때까지 진행
        for k in range(potential_sleep_start_index, num_logs + 1):
            # 디버깅 로그 (특정 날짜에만 출력)
            if debug_date and current_processing_date == debug_date:
                if k < num_logs:
                    print(f"DEBUG [{current_processing_date}] k={k}, is_sleeping={daily_logs_df.iloc[k]['is_sleeping']}, streak={is_false_streak_count}")
                else:
                    print(f"DEBUG [{current_processing_date}] k={k} (data end), streak={is_false_streak_count}")


            current_is_sleeping = False # k == num_logs 일 때를 대비한 기본값
            if k < num_logs: # 실제 데이터가 있는 경우
                current_is_sleeping = daily_logs_df.iloc[k]['is_sleeping']

            if current_is_sleeping:
                if is_false_streak_count >= break_min_logs and is_false_streak_count < awakening_min_logs:
                    # False 스트릭이 브레이크 조건 만족 후 True로 바뀜 -> 브레이크 카운트
                    breaks_in_chunk += 1
                is_false_streak_count = 0 # True이면 리셋
            else: # is_sleeping == False 또는 k == num_logs
                is_false_streak_count += 1
            
            # '완전 깸' 또는 '데이터 끝' 판단
            # is_false_streak_count가 awakening_min_logs에 도달했거나, k가 데이터 끝을 가리킬 때
            # (k==num_logs일때는 current_is_sleeping=False로 간주되어 is_false_streak_count가 1 증가된 상태)
            if is_false_streak_count >= awakening_min_logs or k == num_logs:
                # --- 잠 청크(수면 사이클) 종료 ---
                idx_false_started = -1
                if k == num_logs: # 데이터 끝에 도달
                    # 이 경우, is_false_streak_count는 (num_logs-1)까지의 False 연속을 반영
                    # (k==num_logs에서 is_false_streak_count가 1 더 증가했으므로 -1 해줌)
                    idx_false_started = (k - 1) - (is_false_streak_count -1) + 1 if is_false_streak_count > 0 else k
                else: # awakening_min_logs 이상 False 지속
                    idx_false_started = k - is_false_streak_count + 1
                
                actual_woke_up_time_index = max(potential_sleep_start_index, idx_false_started)
                
                if actual_woke_up_time_index >= num_logs :
                    # 이 Warning은 k=num_logs이고 streak=0 (또는 매우 작음)일 때 발생 가능
                    # 즉, 하루가 끝날 때까지 자고 있었거나, 짧게 깨고 끝난 경우.
                    # 이 경우는 정상일 수 있으므로, 로깅 레벨을 낮추거나 조건부로만 출력.
                    if debug_date and current_processing_date == debug_date:
                         print(f"INFO [{current_processing_date}]: actual_woke_up_time_index adjusted. k={k}, streak={is_false_streak_count-1 if k==num_logs else is_false_streak_count}, num_logs={num_logs}, pot_start={potential_sleep_start_index}, false_start={idx_false_started}")
                    actual_woke_up_time_index = num_logs -1 # 마지막 유효 인덱스
                    if actual_woke_up_time_index < 0 : actual_woke_up_time_index = 0 #혹시 모를 상황 대비

                actual_woke_up_time = daily_logs_df.iloc[actual_woke_up_time_index]['created_at']

                # 분석할 로그 범위: '실제 잠듦 시작 시간'부터 '실제 깬 시간' 직전까지
                start_idx_for_logs = potential_sleep_start_index # '잠듦' 판단 윈도우 시작 인덱스
                
                # actual_woke_up_time_index는 '깸'이 시작된 로그의 인덱스. 그 직전까지가 잠든 기간.
                chunk_logs_for_analysis = daily_logs_df.iloc[start_idx_for_logs : actual_woke_up_time_index]
                
                if chunk_logs_for_analysis.empty and not (start_idx_for_logs == actual_woke_up_time_index) : # 비어있지 않아야 함 (시작==끝 제외)
                     if debug_date and current_processing_date == debug_date:
                        print(f"Warning [{current_processing_date}]: chunk_logs_for_analysis is empty. start_idx={start_idx_for_logs}, woke_idx={actual_woke_up_time_index}")
                     current_log_index = k + 1 if k < num_logs else num_logs
                     break 

                total_chunk_duration_obj = actual_woke_up_time - chunk_actual_start_time
                
                if total_chunk_duration_obj < timedelta(minutes=min_valid_chunk_duration_minutes):
                    current_log_index = k + 1 if k < num_logs else num_logs
                    break 

                actual_sleep_in_chunk_count = chunk_logs_for_analysis['is_sleeping_int'].sum()
                actual_sleep_duration_obj = timedelta(seconds=int(actual_sleep_in_chunk_count * log_interval_seconds))
                
                total_duration_sec = total_chunk_duration_obj.total_seconds()
                efficiency = 0
                if total_duration_sec > 0: # 0으로 나누기 방지
                    efficiency = (actual_sleep_duration_obj.total_seconds() / total_duration_sec) * 100
                
                env_logs = chunk_logs_for_analysis
                avg_temp = env_logs['temperature'].mean()
                avg_humidity = env_logs['humidity'].mean()
                avg_brightness = env_logs['brightness'].mean()
                avg_white_noise = env_logs['white_noise_level'].mean()
                week_val = env_logs['week'].iloc[0] if not env_logs.empty else None
                
                sleep_cycles.append({
                    'cycle_start_time': chunk_actual_start_time,
                    'cycle_end_time': actual_woke_up_time,
                    'total_cycle_duration_minutes': round(max(0,total_duration_sec) / 60, 2), # 음수 방지
                    'actual_sleep_duration_minutes': round(actual_sleep_duration_obj.total_seconds() / 60, 2),
                    'sleep_efficiency_percent': round(efficiency, 2),
                    'breaks_count': breaks_in_chunk,
                    'avg_temperature': round(avg_temp, 1) if pd.notna(avg_temp) else None,
                    'avg_humidity': round(avg_humidity, 1) if pd.notna(avg_humidity) else None,
                    'avg_brightness': round(avg_brightness, 1) if pd.notna(avg_brightness) else None,
                    'avg_white_noise_level': round(avg_white_noise, 0) if pd.notna(avg_white_noise) else None,
                    'week': int(week_val) if pd.notna(week_val) else None
                })
                current_log_index = k + 1 if k < num_logs else num_logs # 다음 탐색 시작 위치 (k 다음부터)
                break # 현재 청크 처리 완료, 다음 청크 찾기 위해 외부 while 루프로

            # 브레이크 판단 로직 수정: is_sleeping == True로 돌아올 때 카운트
            # (위의 if current_is_sleeping: 블록에서 처리됨)

        if potential_sleep_start_index != -1 and not sleep_cycles : # 잠은 들었는데 사이클 못만들고 루프 끝난경우
             # 이 경우는 for k 루프가 break 없이 끝났다는 의미 (즉, 완전 깸을 못찾음)
             # 이는 while 루프의 종료 조건(current_log_index < num_logs)에 의해 처리될 것.
             # 만약 while 루프가 current_log_index 업데이트 없이 계속 돌면 무한루프 가능성 -> current_log_index 업데이트 보장 필요.
             # 위에서 current_log_index = k + 1 로 업데이트 되므로 괜찮을 것.
             pass


    return sleep_cycles

In [9]:
# 특정 아기, 특정 날짜의 데이터로 analyze_sleep_cycle_details_v2 테스트
# 예: baby_id=1, 첫째 날 데이터
if not df_realistic_dummy.empty:
    baby1_df = df_realistic_dummy[df_realistic_dummy['baby_id'] == 1].copy()
    if not pd.api.types.is_datetime64_any_dtype(baby1_df['created_at']):
        baby1_df.loc[:, 'created_at'] = pd.to_datetime(baby1_df['created_at'])
    
    if not baby1_df.empty:
        first_day_str = simulator.start_date.strftime('%Y-%m-%d')
        baby1_first_day_df = baby1_df[baby1_df['created_at'].dt.strftime('%Y-%m-%d') == first_day_str]

        if not baby1_first_day_df.empty:
            print(f"\nAnalyzing first day data for baby 1 ({len(baby1_first_day_df)} logs):")
            cycles = analyze_sleep_cycle_details_v2( # 이전 대화의 v2 함수 사용
                baby1_first_day_df,
                debug_date=simulator.start_date.date() # 디버깅 출력용
            )
            if cycles:
                df_cycles = pd.DataFrame(cycles)
                print(df_cycles)
            else:
                print("No sleep cycles found for the first day.")
        else:
            print("No data for the first day of baby 1.")
    else:
        print("No data for baby 1.")


Analyzing first day data for baby 1 (8640 logs):
DEBUG [2024-05-27] k=0, is_sleeping=False, streak=0
DEBUG [2024-05-27] k=1, is_sleeping=False, streak=1
DEBUG [2024-05-27] k=2, is_sleeping=True, streak=2
DEBUG [2024-05-27] k=3, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=4, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=5, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=6, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=7, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=8, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=9, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=10, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=11, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=12, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=13, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=14, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=15, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=16, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=17, is_sleeping=True, streak=0
DEBUG [2024-05-27] k=1